In [1]:
import json
import os
import random
import numpy as np
from utils import smiles_utils
from utils import polysmiles
from utils import base_units
import hoomd
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import foyer
from foyer import Forcefield
import py3Dmol
import ele
import warnings
import operator
from collections import namedtuple
import matplotlib.pyplot as plt
import scipy.optimize
from scipy.special import gamma

warnings.filterwarnings('ignore')
units = base_units.base_units()

In [9]:
class Simulation():
    def __init__(self,
                 system,
                 target_box=None,
                 r_cut = 1.2,
                 e_factor = 0.5,
                 tau = 0.1,
                 dt = 0.0001,
                 auto_scale = True,
                 ref_units = None,
                 mode = "gpu",
                 gsd_write = 1e4,
                 log_write = 1e3,
                 seed = 42
                 ):
        
        self.system = system 
        self.system_pmd = system.system_pmd # Parmed structure
        self.r_cut = r_cut
        self.e_factor = e_factor
        self.tau = tau
        self.dt = dt
        self.auto_scale = auto_scale
        self.ref_units = ref_units
        self.mode = mode
        self.gsd_write = gsd_write
        self.log_write = log_write
        self.seed = seed

        if ref_units and not auto_scale:
            self.ref_energy = ref_units['energy']
            self.ref_distance = ref_units['distance']
            self.ref_mass = ref_units['mass']           
        
        elif auto_scale and not ref_units: # Pulled from mBuild hoomd_simulation.py
            self.ref_mass = max([atom.mass for atom in self.system_pmd.atoms])
            pair_coeffs = list(set((atom.type,
                                    atom.epsilon,
                                    atom.sigma) for atom in self.system_pmd.atoms))
            self.ref_energy = max(pair_coeffs, key=operator.itemgetter(1))[1]
            self.ref_distance = max(pair_coeffs, key=operator.itemgetter(2))[2]
            
        self.reduced_target_L = self.system.target_L / self.ref_distance
        self.reduced_init_L = self.system_pmd.box[0] / self.ref_distance
        if target_box:
            self.target_box = target_box
        else:
            self.target_box = [self.reduced_target_L]*3
            
        self.log_quantities = [ 
        "temperature",
        "pressure",
        "volume",
        "potential_energy",
        "kinetic_energy",
        "pair_lj_energy",
        "bond_harmonic_energy",
        "angle_harmonic_energy"
        ]
    

    def quench(self, kT, n_steps, shrink_kT=10, shrink_steps=1e6):
        '''
        '''
        # Get hoomd stuff set:
        create_hoomd_simulation(self.system_pmd, self.ref_distance,
                                self.ref_mass, self.ref_energy,
                                self.r_cut, self.auto_scale)
        _all = hoomd.group.all()
        hoomd.md.integrate.mode_standard(dt=self.dt)
        integrator = hoomd.md.integrate.nvt(group=_all, kT=shrink_kT, tau=self.tau) # shrink temp
        integrator.randomize_velocities(seed=self.seed)
        
        # Set up shrinking box_updater:
        shrink_gsd = hoomd.dump.gsd("trajectories/traj-shrink.gsd",
                       period=self.gsd_write, group=_all, phase=0, overwrite=True)
        x_variant = hoomd.variant.linear_interp([(0, self.reduced_init_L),
                                                 (shrink_steps, self.target_box[0]*10)])
        y_variant = hoomd.variant.linear_interp([(0, self.reduced_init_L),
                                                 (shrink_steps, self.target_box[1]*10)])
        z_variant = hoomd.variant.linear_interp([(0, self.reduced_init_L),
                                                 (shrink_steps, self.target_box[2]*10)])
        box_updater = hoomd.update.box_resize(Lx = x_variant, Ly = y_variant, Lz = z_variant)
        
        # Run shrink portion of simulation:
        hoomd.run_upto(shrink_steps)
        shrink_gsd.disable()
        box_updater.disable()
        
        # Start new log and gsd files:
        hoomd.dump.gsd("trajectories/sim_traj.gsd",
                       period=self.gsd_write,
                       group=_all,
                       phase=0,
                       overwrite=True)
        hoomd.analyze.log("logs/sim_traj.log",
                          period=self.log_write,
                          quantities = self.log_quantities,
                          header_prefix="#",
                          overwrite=True, phase=0)
        # Run primary simulation:
        integrator.set_params(kT=kT)
        integrator.randomize_velocities(seed=self.seed)
        hoomd.run(n_steps+shrink_steps)
        
        
    def anneal(self,
              kT_init=None,
              kT_final=None,
              step_sequence=None,
              schedule=None,
              shrink_kT=10,
              shrink_steps=1e6
              ):
        
        if not schedule:
            temps = np.linspace(kT_init, kT_final, len(step_sequence))
            temps = [np.round(t, 1) for t in temps]
            schedule = dict(zip(temps, step_sequence))          
        
        # Get hoomd stuff set:
        create_hoomd_simulation(self.system_pmd, self.ref_distance,
                                self.ref_mass, self.ref_energy,
                                self.r_cut, self.auto_scale)
        _all = hoomd.group.all()
        hoomd.md.integrate.mode_standard(dt=self.dt)
        integrator = hoomd.md.integrate.nvt(group=_all, kT=shrink_kT, tau=self.tau) # shrink temp
        integrator.randomize_velocities(seed=self.seed)
        
        # Set up shrinking box_updater:
        shrink_gsd = hoomd.dump.gsd("trajectories/traj-shrink.gsd",
                       period=self.gsd_write, group=_all, phase=0, overwrite=True)
        x_variant = hoomd.variant.linear_interp([(0, self.reduced_init_L),
                                                 (shrink_steps, self.target_box[0]*10)])
        y_variant = hoomd.variant.linear_interp([(0, self.reduced_init_L),
                                                 (shrink_steps, self.target_box[1]*10)])
        z_variant = hoomd.variant.linear_interp([(0, self.reduced_init_L),
                                                 (shrink_steps, self.target_box[2]*10)])
        box_updater = hoomd.update.box_resize(Lx = x_variant, Ly = y_variant, Lz = z_variant)
        
        hoomd.run_upto(shrink_steps)
        print('Shrink step finished.')
        self.shrink_gsd.disable()
        box_updater.disable()
        print('Anneal Schedule:')
        print(schedule)
        print('-------------------------------')
        
        # Start annealing steps:
        anneal_gsd = hoomd.dump.gsd("trajectories/traj-anneal.gsd",
                                   period=self.gsd_write,
                                   group=_all,
                                   phase=0,
                                   overwrite=True)
        
        hoomd.analyze.log("logs/sim_traj.log",
                          period=self.log_write,
                          quantities = self.log_quantities,
                          header_prefix="#",
                          overwrite=True, phase=0)
        
        last_time_step = shrink_steps
        for kT in schedule:
            print('Running @ Temp = {}'.format(kT))
            n_steps = schedule[kT]
            print('Running for {} steps'.format(n_steps))
            integrator.set_params(kT=kT)
            integrator.randomize_velocities(seed=self.seed)
            hoomd.run(last_time_step + n_steps)
            last_time_step += n_steps
            print('Current time step = {}'.format(last_time_step))
            print('-----------------------------------------------')
            print()
        

class System():
    
    def __init__(self,
                 molecule,
                 para_weight,
                 density,
                 n_compounds=None,
                 polymer_lengths=None,
                 forcefield=None,
                 epsilon=1e-7,
                 sample_pdi=False,
                 pdi=None,
                 Mn=None,
                 Mw=None,
                 mass_dist_type='weibull',
                 remove_hydrogens=False,
                 assert_dihedrals=True,
                 seed=24
                ):
        self.molecule = molecule
        self.para_weight = para_weight
        self.density = density
        self.target_L = None
        self.remove_hydrogens = remove_hydrogens
        self.epsilon = epsilon
        self.forcefield = forcefield
        self.assert_dihedrals = assert_dihedrals
        self.seed = seed
        self.system_mass = 0
        self.para = 0 # keep track for now to check things are working, maybe keep?
        self.meta = 0
        
        if sample_pdi:
            if isinstance(n_compounds, int):
                self.n_compounds = n_compounds
            elif isinstance(n_compounds, list) and len(n_compounds) == 1:
                self.n_compounds = n_compounds[0]
            elif isinstance(n_compounds, list) and len(n_compounds) != 1:
                raise TypeError('n_compounds should be of length 1 when sample_pdi is True.')
            pdi_arg_sum = sum([x is not None for x in [pdi, Mn, Mw]])
            assert pdi_arg_sum >= 2, 'At least two of [pdi, Mn, Mw] must be given.'
            if pdi_arg_sum == 3:
                #special case, make sure that pdi = Mw / Mn
                assert abs(pdi - (Mw/Mn)) < self.epsilon, 'PDI value does not match Mn and Mw values.'
            else:
                # need to recover one of Mw or Mn or pdi
                if Mn is None:
                    Mn = Mw / pdi
                if Mw is None:
                    Mw = pdi * Mn
                if pdi is None:
                    pdi = Mw / Mn
            self.Mn = Mn
            self.Mw = Mw
            self.pdi = pdi
                    
            # this returns a numpy.random callable set up with recovered parameters
            mass_distribution_dict = self._recover_mass_dist(mass_dist_type)
            self.mass_sampler = mass_distribution_dict['sampler']
            self.mass_distribution = mass_distribution_dict['functional_form']
            # TODO: make sure we don't sample any negative weights
            samples = np.round(self.mass_sampler(n_compounds)
                              ).astype(int)
            # get all unique lengths in increasing order
            self.polymer_lengths = sorted(list(set(samples)))
            # get count of each length
            self.n_compounds = [list(samples).count(x) for x in self.polymer_lengths]
            print(f'polymer_lengths: {self.polymer_lengths}, n_compounds: {self.n_compounds}')
            
        else: # Do some validation, get things in the correct data types
            if not isinstance(n_compounds, list):
                self.n_compounds = [n_compounds]
            else:
                self.n_compounds = n_compounds

            if not isinstance(polymer_lengths, list):
                self.polymer_lengths = [polymer_lengths]
            else:
                self.polymer_lengths = polymer_lengths
        
        if len(self.n_compounds) != len(self.polymer_lengths):
            raise ValueError('n_compounds and polymer_lengths should be equal length')
        
        self.system_mb = self._pack() # mBuild object before applying FF
        if self.forcefield:
            self.system_pmd = self._type_system() # parmed object after applying FF
        
    def _weibull_k_expression(self, x):
        return (2. * x * gamma(2./x)) / gamma(1./x)**2 - (self.Mw / self.Mn)
    
    def _weibull_lambda_expression(self, k):
        return self.Mn * k / gamma(1./k)
    
    def _recover_mass_dist(self, distribution='Gaussian'):
        '''This function takes in two of the three quantities [Mn, Mw, PDI],
           and fits either a Gaussian or Weibull distribution of molar masses to them.'''
        if distribution.lower() != 'gaussian' and distribution.lower() != 'weibull':
            raise(ValueError('Molar mass distribution must be either "gaussian" or "weibull".'))
        if distribution.lower() == 'gaussian':
            mean = self.Mn
            sigma = self.Mn * (self.Mw - self.Mn)
            return {'sampler': lambda N: np.random.normal(loc=mean, scale=sigma, size=N),
                    'functional_form': lambda x: np.exp(-(x-Mn)**2 / (2. * sigma))}
        elif distribution.lower() == 'weibull':
            # get the shape parameter
            a = scipy.optimize.root(self._weibull_k_expression, x0=1.)
            recovered_k = a['x']
            # get the scale parameter
            recovered_lambda = self._weibull_lambda_expression(recovered_k)
            return {'sampler': lambda N: recovered_lambda * np.random.weibull(recovered_k, size=N),
                    'functional_form': lambda x: recovered_k / recovered_lambda * (x / recovered_lambda) ** (recovered_k - 1) * np.exp(- (x / recovered_lambda) ** recovered_k)}
        
    def _pack(self, box_expand_factor=5):
        random.seed(self.seed)
        mb_compounds = []
        for _length, _n in zip(self.polymer_lengths, self.n_compounds):
            for i in range(_n):
                polymer, sequence = build_molecule(self.molecule, _length,
                                        self.para_weight)

                mb_compounds.append(polymer)
                self.para += sequence.count('para')
                self.meta += sequence.count('meta')
            mass = _n * np.sum(ele.element_from_symbol(p.name).mass for p in polymer.particles())
            self.system_mass += mass # amu
        
        # Figure out correct box dimensions and expand the box to make the PACKMOL step faster
        # Will shrink down to accurate L during simulation
        L = self._calculate_L() * box_expand_factor
        system = mb.packing.fill_box(
            compound = mb_compounds,
            n_compounds = [1 for i in mb_compounds],
            box=[L, L, L],
            overlap=0.2,
            edge=0.9,
            fix_orientation=True)
        system.Box = mb.box.Box([L, L, L])
        return system
    
    
    def _type_system(self):
        if self.forcefield == 'gaff':
            forcefield = foyer.forcefields.load_GAFF()
        elif self.forcefield == 'opls':
            forcefield = foyer.Forcefield(name='oplsaa')
        
        typed_system = forcefield.apply(self.system_mb,
                                       assert_dihedral_params=self.assert_dihedrals)
        if self.remove_hydrogens: # not sure how to do this with Parmed yet
            removed_hydrogen_count = 0 # subtract from self.mass
            pass    
        return typed_system
    
    def _calculate_L(self):
        '''
        Calcualte the box length needed for entered density
        Right now, assuming cubic box
        Return L in nm (mBuild units)
        '''
        M = self.system_mass * units["amu_to_g"] # grams
        L = (M / self.density)**(1/3) # centimeters
        L *= units['cm_to_nm'] # convert cm to nm
        self.target_L = L # Used during shrink step
        return L

def build_molecule(molecule, length, para_weight):
    '''
    `build_molecule` uses SMILES strings to build up a polymer from monomers.
    The configuration of each monomer is determined by para_weight and the
    random_sequence() function.  
    Uses DeepSMILES behind the scenes to build up SMILES string for a polymer.
    
    Parameters
    ----------
    molecule : str
        The monomer molecule to be used to build up the polymer.
        Available options are limited  to the .json files in the compounds directory
        Use the molecule name as seen in the .json file without including .json
    length : int
        The number of monomer units in the final polymer molecule
    para_weight : float, limited to values between 0 and 1
        The relative amount of para configurations compared to meta.
        Passed into random_sequence() to determine the monomer sequence of the polymer.
        A 70/30 para to meta system would require a para_weight = 0.70
    
    Returns
    -------
    molecule_string_smiles : str
        The complete SMILES string of the polymer molecule
    '''
    f = open('compounds/{}.json'.format(molecule))
    mol_dict = json.load(f)    
    f.close()
    monomer_sequence = random_sequence(para_weight, length)
    molecule_string = '{}'

    for idx, config in enumerate(monomer_sequence):
        if idx == 0: # append template, but not brackets
            monomer_string = mol_dict['{}_template'.format(config)]
            if molecule == 'PEEK':
                monomer_string = "O" + monomer_string[1:]
            molecule_string = molecule_string.format(monomer_string)
            if len(monomer_sequence) == 1:
                molecule_string = molecule_string.replace('{}', '')
                continue

        elif idx == length - 1: # Don't use template for last iteration
            brackets = polysmiles.count_brackets(mol_dict['{}_deep_smiles'.format(config)])
            monomer_string = mol_dict['{}_deep_smiles'.format(config)]
            molecule_string = molecule_string.format(monomer_string, brackets)

        else: # Continue using template plus brackets
            brackets = polysmiles.count_brackets(mol_dict['{}_deep_smiles'.format(config)])
            monomer_string = mol_dict['{}_template'.format(config)]
            molecule_string = molecule_string.format(monomer_string, brackets)
    
    molecule_string_smiles = smiles_utils.convert_smiles(deep = molecule_string)
    compound = mb.load(molecule_string_smiles, smiles=True)
    return compound, monomer_sequence


def random_sequence(para_weight, length):
    '''
    random_sequence returns a list containing a random sequence of strings 'para' and 'meta'.
    This is used by build_molecule() to create a complete SMILES string of a molecule.
    
    Parameters:
    -----------
    para_weight : float, limited to values between 0 and 1
        The relative amount of para configurations compared to meta.
        Defined in build_molecule()
    length : int
        The number of elements in the random sequence.
        Defined in build_molecule()
    '''
    meta_weight = 1 - para_weight
    options = ['para', 'meta']
    probability = [para_weight, meta_weight]
    sequence = random.choices(options, weights=probability, k=length)
    return sequence

In [14]:
test_system = System(molecule="PEEK", para_weight=0.60,
                    density=1, n_compounds=[5], sample_pdi=True,
                     pdi=1.01, Mn=14, mass_dist_type='weibull',
                    forcefield=None)

polymer_lengths: [13, 14, 15], n_compounds: [1, 3, 1]


In [37]:
simulation = Simulation(system=test_system)

AttributeError: 'System' object has no attribute 'system_pmd'

In [5]:
simulation.quench(kT=1.5, n_steps=1e6)

HOOMD-blue 2.9.3 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/17/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 860 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 2 exclusions             : 5
notice(2): Particles with 3 exclusions             : 330
notice(2): Particles with 4 exclusions             : 5
notice(2): Particles with 6 exclusions             : 45
notice(2): Particles with 7 exclusions             : 330
notice(2): Particles with 8 exclusions             : 95
notice(2): Particles with 9 exclusions             : 50
notice(2): Neighbors included by diameter       

**ERROR**: Pair type ca-ho not found in pair coeff
**ERROR**: Not all force coefficients are set


No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions
HOOMD SimulationContext updated from ParmEd Structure


RuntimeError: Error updating force coefficients

In [ ]:
hoomd.dump.gsd()

In [ ]:
simulation.anneal(kT_init=2, kT_final=1, step_sequence=[1e5, 1e5, 1e5, 1e5])

In [ ]:
##############################################
##OLD STUFF
##############################################

In [ ]:
def build_system(packing='bcc'):
    '''
    Generate a simple LJ particle system using hoomd's create_lattice function
    packing 
    '''
    hoomd.context.initialize("")
    if packing == 'fcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.fcc(a=1.58)), n=6)
    elif packing == 'bcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.bcc(a=1.29)), n=6)
    hoomd.dump.gsd('{}_system.gsd'.format(packing), group = hoomd.group.all(), period=None, overwrite=True)
    return system

def hoomd_simulation(system, temp, tau):  
    nl = hoomd.md.nlist.cell()
    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
    lj.pair_coeff.set('A', 'A', alpha=1.0, epsilon=1.0, sigma=1.0)
    hoomd.md.integrate.mode_standard(dt=0.001)
    _all = hoomd.group.all()
    nvt = hoomd.md.integrate.nvt(group=_all, kT=temp, tau=tau)
    nvt.randomize_velocities(seed=23)
    hoomd.analyze.log(filename='{}-tau_out.log'.format(tau),
                      quantities=["time", "temperature", "potential_energy"],
                      period=100,
                      overwrite=True
                     )
    #hoomd.dump.gsd('tau-trajectory.gsd', period=5e3, group=_all, overwrite=True)
    hoomd.run(3e5)

# --------------------------------------------------------

PEEK:  
1 amu = 1.66054e-24 g  
1.32 g/cm^3  
1.32 g/nm^3  
C19H12O3  
monomer_amu = 288.302  

In [ ]:
def build_peek_system(num_mols, poly_length, density):
    '''
    This function uses mBuild's packing functionality to create a very low dense system of molecules
    to allow for easier packing. A short simulation is then ran using HoomD to shrink the system to the
    desired starting density.
    '''
    peek_poly_smi = polysmiles.poly_smiles('occccC=O)ccccOcc*c*ccc6)))))))cc6)))))))cc6',
                                      length=poly_length)
    peek_poly = mb.load(peek_poly_smi, smiles=True)
    
    mol_amu = poly_length * 288.302
    mol_grams = mol_amu * 1.66054e-24
    system_mass = mol_grams * num_mols
    edge_cm = (system_mass / density)**(1/3)
    edge_nm = edge_cm * 1e7

    init_box = mb.Box([edge_nm*5]*3)
    system = mb.fill_box(peek_poly, num_mols, init_box)
    
    GAFF = foyer.forcefields.load_GAFF()
    system_pmd = GAFF.apply(system)
    create_hoomd_simulation(system_pmd, r_cut=1.2, auto_scale=True)
    forces = [f for f in hoomd.context.current.forces 
                if not isinstance(f, hoomd.md.charge.pppm)]
    
    hoomd.context.current.forces = forces
    _all = hoomd.group.all()
    hoomd.md.integrate.mode_standard(dt=0.0001)
    integrator = hoomd.md.integrate.nvt(group=_all, kT=2.0, tau=0.1)
    hoomd.dump.gsd("trajectories/start-shrink.gsd", period=None, group=_all, overwrite=True)
    hoomd.dump.gsd("trajectories/traj-shrink.gsd", period=1e5, group=_all, phase=0, overwrite=True)
    integrator.randomize_velocities(seed=42);

    hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,edge_nm*50), (1e6,edge_nm*10)], zero=0))
    hoomd.run(1e6)
    gsd_restart = hoomd.dump.gsd("trajectories/out-shrink.gsd", period=None, group=_all, overwrite=True)
    gsd_restart.write_restart()

In [ ]:
build_peek_system(num_mols=20, poly_length=5, density=1.2)

# -------------------------------------------------------------------------

In [ ]:
# Smiles Strings
'''
DEEPSMILES:
PEEK-para = "occccoccccC=O)cccccc6)))))))cc6)))))))cc6"
PEEK-meta = "occcoccccC=O)cccccc6)))))))cc6)))))))ccc6"
PEKK-para = "ccccoccccC=o)ccccC=o))cc6)))))))cc6)))))))cc6"
PEKK-meta = "ccccoccccC=o)cccC=o))ccc6)))))))cc6)))))))cc6"

SMILES:
PEEK-para = "oc3ccc(oc2ccc(C(=O)c1ccccc1)cc2)cc3"
PEEK-meta = "oc3cc(oc2ccc(C(=O)c1ccccc1)cc2)ccc3"
PEKK-para = "c3ccc(oc2ccc(C(=o)c1ccc(C=o)cc1)cc2)cc3"
PEKK-meta = "c3ccc(oc2ccc(C(=o)c1cc(C=o)ccc1)cc2)cc3"
'''


# DeepSMILES with bonded atom indicated
'''
PEEK Para Bonded:
"occccoccccC=O)cc*c*ccc6)))))))cc6)))))))cc6"
"occccoccccC=O)ccc*c*cc6)))))))cc6)))))))cc6"
"occccoccccC=O)cccc*c*c6)))))))cc6)))))))cc6"

PEEK Meta Bonded:
"occcoccccC=O)ccc*c*cc6)))))))cc6)))))))ccc6"
"occcoccccC=O)cc*c*ccc6)))))))cc6)))))))ccc6"
"occcoccccC=O)cccc*c*c6)))))))cc6)))))))ccc6"

PEKK Para Bonded
"ccccoccccC=o)cccc*C*=o))cc6)))))))cc6)))))))cc6"
PEKK Meta Bonded
"ccccoccccC=o)ccc*C*=o))ccc6)))))))cc6)))))))cc6"
'''


# Build up dicts and save to json file
'''
PEKK_dict = {}

PEKK_dict['para_deep_smiles'] = "ccccoccccC=o)ccccC=o))cc6)))))))cc6)))))))cc6"
PEKK_dict['meta_deep_smiles'] = "ccccoccccC=o)cccC=o))ccc6)))))))cc6)))))))cc6"
PEKK_dict['para_smiles'] = "c3ccc(oc2ccc(C(=o)c1ccc(C=o)cc1)cc2)cc3"
PEKK_dict['meta_smiles'] = "c3ccc(oc2ccc(C(=o)c1cc(C=o)ccc1)cc2)cc3"
PEKK_dict['para_template'] = "ccccoccccC=o)cccc*C*=o))cc6)))))))cc6)))))))cc6"
PEKK_dict['meta_template'] = "ccccoccccC=o)ccc*C*=o))ccc6)))))))cc6)))))))cc6"
PEKK_dict['monomer_mass'] = 302.329


PEEK_dict = {}

PEEK_dict['para_deep_smiles'] = "occccoccccC=O)cccccc6)))))))cc6)))))))cc6"
PEEK_dict['meta_deep_smiles'] = "occcoccccC=O)cccccc6)))))))cc6)))))))ccc6"
PEEK_dict['para_smiles'] = "oc3ccc(oc2ccc(C(=O)c1ccccc1)cc2)cc3"
PEEK_dict['meta_smiles'] = "oc3cc(oc2ccc(C(=O)c1ccccc1)cc2)ccc3"
PEEK_dict['para_template'] = "occccoccccC=O)ccc{}{}ccc6)))))))cc6)))))))cc6"
PEEK_dict['meta_template'] = "occcoccccC=O)cccc{}{}cc6)))))))cc6)))))))ccc6"
PEEK_dict['monomer_mass'] = 288.302



file_name = 'compounds/PEKK.json'
with open(file_name, 'w') as fp:
    json.dump(PEKK_dict, fp)
'''